In [30]:
import pandas as pd
import numpy as np
import sklearn

In [31]:
# read train,val and test from files

In [32]:
train = pd.read_csv('input/train.tsv', sep='\t')
val = pd.read_csv('input/val.tsv', sep='\t')
test = pd.read_csv('input/test.tsv', sep='\t')

In [33]:
X_train = train.iloc[:,1:152]

In [34]:
y_train = train.loc[:,"target"]

In [35]:
X_test = val.iloc[:,1:152]

In [36]:
y_test = val.loc[:,"target"]

In [45]:
X_train.head()

,graph_dim_1,graph_dim_2,graph_dim_3,graph_dim_4,graph_dim_5,graph_dim_6,graph_dim_7,graph_dim_8,graph_dim_9,graph_dim_10,...,graph_dim_142,graph_dim_143,graph_dim_144,graph_dim_145,graph_dim_146,graph_dim_147,graph_dim_148,graph_dim_149,graph_dim_150,user_id
0,-0.188908,0.301332,0.252131,-0.323816,-0.440463,0.015726,-0.175798,0.371103,0.219573,-0.260040,...,-0.205058,0.001048,-0.073120,0.003778,0.277356,0.257582,-0.352065,-0.023871,-0.049963,1448635298
1,0.195424,0.384668,-0.122488,-0.164691,-0.290291,0.278738,0.015823,-0.143936,0.039850,-0.007902,...,-0.146156,0.012703,0.299493,0.124290,-0.316433,-0.095917,-0.085738,0.092192,0.070166,1095419257
2,-0.142850,-0.003906,0.329644,-0.112234,-0.054298,-0.161891,-0.137229,-0.353879,-0.166023,-0.195910,...,0.083953,0.161124,0.264300,-0.338039,0.039093,0.275086,0.173041,-0.617767,0.176780,1437313259602857985
3,0.138840,0.193202,0.052293,-0.116355,-0.211692,0.285736,-0.072009,-0.309909,0.330486,-0.473953,...,0.087964,-0.097014,-0.061762,-0.438787,-0.178216,0.142492,-0.104245,0.361036,-0.524681,838725492916224000
4,-0.072190,-0.081853,0.137990,-0.102708,-0.251578,0.116002,-0.021854,-0.131041,0.175357,0.103007,...,0.359756,-0.158123,-0.221713,0.338095,-0.066974,0.022784,-0.396275,-0.270856,0.034927,1152247887587184640


In [30]:
X_test

,graph_dim_1,graph_dim_2,graph_dim_3,graph_dim_4,graph_dim_5,graph_dim_6,graph_dim_7,graph_dim_8,graph_dim_9,graph_dim_10,...,graph_dim_142,graph_dim_143,graph_dim_144,graph_dim_145,graph_dim_146,graph_dim_147,graph_dim_148,graph_dim_149,graph_dim_150,user_id
0,0.190901,0.131550,-0.112169,0.166876,-0.000311,-0.023343,-0.129785,0.006558,0.029834,0.196156,...,0.144383,-0.376118,-0.429327,-0.139352,-0.163238,-0.250924,-0.000699,-0.394026,-0.112814,1493590883954044938
1,-0.161348,-0.197149,0.097824,-0.526097,0.169401,-0.149939,0.170573,0.113505,-0.097271,-0.124499,...,-0.319094,0.279728,0.036602,0.278061,0.224577,-0.046691,0.129290,0.142409,-0.078959,15632337
2,-0.010724,-0.074530,-0.000211,0.219826,-0.440146,0.174967,-0.389276,-0.007553,0.079228,-0.110118,...,0.362958,0.115667,0.202408,0.013278,-0.142128,0.269984,-0.007029,-0.251939,-0.322457,1187374837397970944
3,0.208510,0.144599,-0.034021,0.017045,-0.101741,0.243353,0.040184,0.134637,0.122690,-0.001832,...,0.162849,0.361891,-0.072816,-0.152048,-0.085703,0.074861,0.074254,-0.101711,-0.334468,955169960363376641
4,0.200732,-0.120989,0.027544,-0.114643,0.343851,0.071952,-0.084613,-0.148303,0.067299,0.325764,...,0.108702,-0.076426,-0.054077,-0.038062,0.007449,0.228312,-0.096890,-0.294227,0.059763,1458425365475508225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33354,0.021267,-0.052472,0.286227,0.252419,0.086489,0.130524,0.227365,-0.053208,0.129457,-0.049920,...,0.175946,0.035965,-0.146107,0.098317,-0.238036,0.181995,-0.056941,-0.233467,0.037802,1498935726582341637
33355,0.070728,0.158180,-0.097002,-0.127935,0.219346,0.050796,0.170390,0.008489,-0.006427,-0.004136,...,-0.134887,-0.324472,-0.016312,0.043725,0.269832,0.109030,-0.102794,-0.405820,-0.016716,1630818589
33356,-0.215520,0.374962,-0.159312,-0.105047,-0.137474,-0.338756,-0.116490,0.000233,0.054003,-0.074802,...,-0.000549,0.373732,-0.293287,-0.211379,0.327570,0.075305,0.121313,-0.221681,0.291414,1349815435
33357,-0.289602,0.240277,0.032482,0.388372,0.150738,0.166351,-0.151222,0.256500,0.021857,-0.067070,...,0.747996,0.137995,0.013351,-0.193227,0.198072,-0.225575,-0.064821,0.235161,0.049892,3232111399


In [24]:
y_train

0         0
1         0
2         0
3         0
4         0
         ..
106740    1
106741    0
106742    0
106743    0
106744    0
Name: target, Length: 106745, dtype: int64

In [32]:
y_test

0        1
1        0
2        0
3        0
4        1
        ..
33354    1
33355    1
33356    0
33357    0
33358    0
Name: target, Length: 33359, dtype: int64

In [ ]:
#Lasso : first solution

In [10]:
# from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
# from sklearn import model_selection
# from sklearn.metrics import mean_squared_error
# %matplotlib inline
# import matplotlib.pyplot as plt

In [46]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

In [58]:
# lasso = Lasso(max_iter=10000, normalize=True)

# lassocv = LassoCV(alphas=None, cv=10, max_iter=100000, normalize=True)
# lassocv.fit(X_train, y_train)
# lasso.set_params(alpha=lassocv.alpha_)
# print("Alpha=", lassocv.alpha_)
# lasso.fit(X_train, y_train)
# print("mse = ",mean_squared_error(y_test, lasso.predict(X_test)))
# print("best model coefficients:")
# pd.Series(lasso.coef_, index=X_train.columns)

Alpha= 4.432332110388478e-07
mse =  0.10985454883417317
best model coefficients:


graph_dim_1     -1.603218e-02
graph_dim_2      1.019848e-02
graph_dim_3     -4.062988e-03
graph_dim_4     -7.143024e-03
graph_dim_5      3.792590e-02
                     ...     
graph_dim_147    3.704284e-02
graph_dim_148   -5.730044e-03
graph_dim_149   -3.864395e-02
graph_dim_150    1.398705e-02
user_id          1.546575e-19
Length: 151, dtype: float64

In [ ]:
#Lasso : second solution

In [60]:
pipeline = Pipeline([
                     ('scaler',StandardScaler()),
                     ('model',Lasso())
])

In [61]:
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.01,1,0.01)},
                      cv = 5, scoring="neg_mean_squared_error",verbose=3
                      )

In [62]:
search.fit(X_train,y_train)

Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV] model__alpha=0.01 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .................. model__alpha=0.01, score=-0.115, total=   0.8s
[CV] model__alpha=0.01 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV] .................. model__alpha=0.01, score=-0.116, total=   0.7s
[CV] model__alpha=0.01 ...............................................


...

[CV] .... model__alpha=0.7100000000000001, score=-0.166, total=   0.7s
[CV] model__alpha=0.7100000000000001 .................................
[CV] .... model__alpha=0.7100000000000001, score=-0.169, total=   0.6s
[CV] model__alpha=0.7100000000000001 .................................
[CV] .... model__alpha=0.7100000000000001, score=-0.168, total=   0.6s
[CV] model__alpha=0.7100000000000001 .................................
[CV] .... model__alpha=0.7100000000000001, score=-0.166, total=   0.6s
[CV] model__alpha=0.7100000000000001 .................................
[CV] .... model__alpha=0.7100000000000001, score=-0.170, total=   0.6s
[CV] model__alpha=0.72 ...............................................
[CV] .................. model__alpha=0.72, score=-0.166, total=   0.6s
[CV] model__alpha=0.72 ...............................................
[CV] .................. model__alpha=0.72, score=-0.169, total=   0.6s
[CV] model__alpha=0.72 ...............................................
[CV] .

[CV] .... model__alpha=0.8200000000000001, score=-0.166, total=   0.7s
[CV] model__alpha=0.8200000000000001 .................................
[CV] .... model__alpha=0.8200000000000001, score=-0.170, total=   0.7s
[CV] model__alpha=0.8300000000000001 .................................
[CV] .... model__alpha=0.8300000000000001, score=-0.166, total=   0.7s
[CV] model__alpha=0.8300000000000001 .................................
[CV] .... model__alpha=0.8300000000000001, score=-0.169, total=   0.7s
[CV] model__alpha=0.8300000000000001 .................................
[CV] .... model__alpha=0.8300000000000001, score=-0.168, total=   0.8s
[CV] model__alpha=0.8300000000000001 .................................
[CV] .... model__alpha=0.8300000000000001, score=-0.166, total=   0.7s
[CV] model__alpha=0.8300000000000001 .................................
[CV] .... model__alpha=0.8300000000000001, score=-0.170, total=   0.7s
[CV] model__alpha=0.8400000000000001 .................................
[CV] .

[CV] .... model__alpha=0.9400000000000001, score=-0.169, total=   0.7s
[CV] model__alpha=0.9400000000000001 .................................
[CV] .... model__alpha=0.9400000000000001, score=-0.168, total=   0.6s
[CV] model__alpha=0.9400000000000001 .................................
[CV] .... model__alpha=0.9400000000000001, score=-0.166, total=   0.6s
[CV] model__alpha=0.9400000000000001 .................................
[CV] .... model__alpha=0.9400000000000001, score=-0.170, total=   0.6s
[CV] model__alpha=0.9500000000000001 .................................
[CV] .... model__alpha=0.9500000000000001, score=-0.166, total=   0.6s
[CV] model__alpha=0.9500000000000001 .................................
[CV] .... model__alpha=0.9500000000000001, score=-0.169, total=   0.7s
[CV] model__alpha=0.9500000000000001 .................................
[CV] .... model__alpha=0.9500000000000001, score=-0.168, total=   0.6s
[CV] model__alpha=0.9500000000000001 .................................
[CV] .

[Parallel(n_jobs=1)]: Done 495 out of 495 | elapsed:  5.4min finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99])},
             scoring='neg_mean_squared_error', verbose=3)

In [ ]:
search.best_params_

In [19]:
# get coefficients from Lasso regression

In [41]:
coefficients = search.best_estimator_.named_steps['model'].coef_

In [43]:
coefficients

array([-0.       ,  0.       ,  0.       , -0.       ,  0.       ,
       -0.       ,  0.       , -0.       , -0.       , -0.       ,
       -0.       , -0.       , -0.       ,  0.       , -0.       ,
       -0.       , -0.       ,  0.       , -0.       ,  0.       ,
       -0.       ,  0.       , -0.       ,  0.       ,  0.       ,
        0.       , -0.       ,  0.       , -0.       ,  0.       ,
        0.       , -0.       ,  0.       , -0.       , -0.       ,
        0.       , -0.       ,  0.       , -0.       , -0.       ,
        0.       , -0.       ,  0.       , -0.       ,  0.       ,
        0.       , -0.       , -0.       , -0.       ,  0.       ,
       -0.       ,  0.       ,  0.       ,  0.       , -0.       ,
       -0.       , -0.       , -0.       ,  0.       , -0.       ,
        0.       , -0.       , -0.       , -0.       ,  0.       ,
       -0.       , -0.       , -0.       ,  0.       ,  0.       ,
       -0.       ,  0.       ,  0.       ,  0.       , -0.    

In [27]:
# The importance of a feature is the absolute value of its coefficient

In [44]:
importance = np.abs(coefficients)

In [29]:
importance

array([0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 